<a href="https://colab.research.google.com/github/mimrancomsats/ProgrammingforAI_FALL25/blob/main/Lab_12_BERT_Feature_Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sentiment Analysis (Text Classification)**
*   **Text Cleaning**
*   **BERT Model (Feature Engineering)**
*   **ML Model**

# **Importing Preprocessing Libraries**

In [ ]:
!pip install contractions

import pandas as pd
import string

import re
import contractions
import nltk
#from nltk.tokenize import word_tokenize
#from nltk.corpus import stopwords
#from nltk.stem import WordNetLemmatizer


#nltk.download('wordnet')
#nltk.download('stopwords')
#nltk.download('punkt_tab')


#stopwords.words('english')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 13.7 MB/s eta 0:00:00


# **Reading Data**

In [ ]:
temp_df = pd.read_csv('/content/drive/MyDrive/Programming for AI_SPRING-25/Labs/IMDB Dataset.csv')
df = temp_df.iloc[:1000]

# **Text Cleaning & Preprocessing**

In [ ]:
def remove_html_tags(text):
    return re.sub(r'<.*?>', '', text)

def remove_url(text):
    return re.sub(r'https?://\S+|www\.\S+', '', text)

def remove_contractions(text):
  expanded_text = contractions.fix(text)
  return expanded_text

def remove_punc(text):
    unwanted = set(string.punctuation + string.digits)
    return ''.join(char for char in str(text) if char not in unwanted)


In [ ]:
df['review'] = df['review'].str.lower()

df['review'] = df['review'].apply(remove_html_tags)

df['review'] = df['review'].apply(remove_url)

#df['review'] = df['review'].apply(remove_contractions)

#df['review'] = df['review'].apply(remove_punc)

#df['review'] = df['review'].apply(word_tokenize)

#df['review'] = df['review'].apply(remove_stopwords)

#df['review'] = df['review'].apply(lemmatize_words)

/tmp/ipython-input-3196021694.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].str.lower()
/tmp/ipython-input-3196021694.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(remove_html_tags)
/tmp/ipython-input-3196021694.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

# **Feature Engineering**

**Target Column Encoding**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.preprocessing import LabelEncoder

X = df['review']
Y = df['sentiment']
#print(X)
#print(Y)

encoder = LabelEncoder()
Y = encoder.fit_transform(Y)

#print(Y)

X_train,X_test,y_train,y_test = train_test_split(df['review'],Y,test_size=0.2,random_state=42)
print(X_train)
print(type(X_train))
print(type(y_train))
print(type(X_test))
print(type(y_test))

29     'war movie' is a hollywood genre that has been...
535    this film is really vile. it plays on the urba...
695    okay this is stupid,they say their not making ...
557    crossfire remains one of the best hollywood me...
836    this film was very well advertised. i am an av...
                             ...                        
106    the performance of every actor and actress (in...
270    clifton webb is one of my favorites. however, ...
860    this production was quite a surprise for me. i...
435    you wear only the best italian suits from arma...
102    i bought this game on an impulse buy from walm...
Name: review, Length: 800, dtype: object
<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>
<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>


**BERT Encodings**

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

# Load BERT preprocessing and encoding models from TensorFlow Hub
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3", trainable=False)

# Define function to get BERT embeddings
def get_bert_embeddings(texts):
    text_input = tf.constant(texts)
    #print(text_input)
    preprocessed_text = preprocessor(text_input)
    #print(preprocessed_text)
    outputs = encoder(preprocessed_text)
    #print(outputs)
    pooled_output = outputs['pooled_output']
    #print(pooled_output)
    return pooled_output

# Get BERT embeddings for training and test sets
train_embeddings = get_bert_embeddings(X_train)
test_embeddings = get_bert_embeddings(X_test)

print(train_embeddings.shape)
print(type(train_embeddings))
print(test_embeddings[1])

(800, 768)
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor(
[-0.84073687 -0.59189045 -0.9097728   0.6310802   0.599395    0.02088368
  0.25635287  0.39981055 -0.8674617  -0.9999894  -0.5830792   0.8910839
  0.98197573  0.3501186   0.8191783  -0.45303473  0.0131651  -0.57301855
  0.4219455   0.5518912   0.6356722   0.99998593  0.03048339  0.33152887
  0.46607465  0.9825646  -0.71296084  0.87485325  0.91680485  0.7459768
 -0.24752344  0.2646369  -0.9815184  -0.29308987 -0.9563796  -0.9892382
  0.3400752  -0.59685224  0.1676921   0.11031386 -0.8166782   0.31128865
  0.9999886   0.16165458  0.40021664 -0.01561543 -0.9999987   0.11431252
 -0.7619638   0.9259451   0.7238823   0.91489303  0.30341366  0.4524649
  0.43652275 -0.29131925 -0.15321332  0.02035416 -0.21815494 -0.57413334
 -0.5073336   0.30865318 -0.84160995 -0.8700797   0.8179334   0.76557964
 -0.13522086 -0.14024183 -0.08753052  0.06679182  0.617325    0.34093443
 -0.33972022 -0.8662558   0.5860027   0.4272908  -0.

**ML Model**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# Initialize and train the Random Forest model
rf_model = RandomForestClassifier()
rf_model.fit(train_embeddings, y_train)

# Predict on the test set
y_pred = rf_model.predict(test_embeddings)

print (accuracy_score(y_test,y_pred))
print (confusion_matrix(y_test,y_pred))

0.695
[[67 37]
 [24 72]]


In [ ]:
  !pip install --upgrade tensorflow_hub

  import tensorflow_hub as hub

  model = hub.KerasLayer("https://tfhub.dev/google/nnlm-en-dim128/2")
  embeddings = model(["The rain in Spain.", "falls",
                      "mainly", "In the plain!"])

  print(embeddings.shape)  #(4,128)
  print(type(embeddings))
  print(embeddings[0])

(4, 128)
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor(
[ 1.59735814e-01 -1.29867673e-01 -1.35819957e-01 -9.24930871e-02
  2.10374016e-02 -7.29679167e-02 -1.11245513e-01 -9.90176424e-02
  1.71865672e-02 -7.84225911e-02  3.15940194e-02  6.22844100e-02
 -8.64286870e-02 -7.39999115e-02  4.68084142e-02  2.64361743e-02
 -1.51468843e-01  8.37246031e-02 -4.84693572e-02  3.62105906e-01
 -4.43154573e-03  8.15866888e-02 -4.09569405e-02 -1.05201520e-01
 -1.68916248e-02  2.99027767e-02  8.40939730e-02  4.29377109e-02
 -3.50504890e-02 -9.36504006e-02 -3.67183425e-02  3.71749699e-02
  2.95477919e-02  1.72754377e-01  1.08686611e-02 -1.57570407e-01
 -3.82928923e-02 -4.58077043e-02 -9.41693187e-02 -2.65682954e-03
 -5.25422022e-03 -3.45592424e-02 -1.69853754e-02  1.99845843e-02
  2.64117979e-02  1.15411691e-02  3.72448415e-02  6.33172691e-02
  5.32621630e-02  1.91026106e-02  6.38045296e-02  5.82209881e-03
 -1.88273005e-03  1.63987521e-02  1.42705115e-02 -4.10080254e-02
  6.77466765e-02